In [9]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

def readData(filename,columns):
    data = pd.read_csv(filename, sep="\t",header=None)
    data.columns = columns
    return data

def expandCol(data,column_name):
    cv = CountVectorizer(tokenizer=lambda x: x.split("/"))
    return cv.fit_transform(data[column_name].astype(str))

def appendExpandedCol(data , vector_to_expand):
    for key in vector_to_expand.keys():
        col_list=[key+str(num) for num in range(vector_to_expand[key].shape[1])]
        data =  pd.concat([data,pd.DataFrame(vector_to_expand[key].toarray(), columns=col_list)],axis=1)
    return data

def getQuestData():
    vector_to_expand = {}    
    question_data = readData('question_info.txt',["Q_id","Q_tag","Word_seq","Q_Char_seq","Likes","Ans","Top_Ans"])
    del question_data["Word_seq"]
    vector_to_expand["Q_tag"] = expandCol(question_data,"Q_tag")
    vector_to_expand["Q_Char_seq"] = expandCol(question_data,"Q_Char_seq")
    del question_data["Q_tag"]
    del question_data["Q_Char_seq"]
    return appendExpandedCol(question_data , vector_to_expand)

def getExpData():
    vector_to_expand = {}    
    expert_data = readData('user_info.txt',["U_id","U_tag","U_Word_seq","U_Char_seq"])
    del expert_data["U_Word_seq"]
    vector_to_expand["U_tag"] = expandCol(expert_data,"U_tag")
    vector_to_expand["U_Char_seq"] = expandCol(expert_data,"U_Char_seq")
    del expert_data["U_tag"]
    del expert_data["U_Char_seq"]
    return appendExpandedCol(expert_data , vector_to_expand)

question_data = getQuestData()
expert_data = getExpData()
expert_question_dist = pd.read_csv('invited_info_train.txt', sep="\t",header=None)
expert_question_dist.columns = ["Q_id","U_id","output"]
final_data = pd.merge(question_data, expert_question_dist, on="Q_id",how = 'inner')
final_data = pd.merge(final_data, expert_data, on='U_id', how='inner')
final_target = final_data["output"]
del final_data["output"]
model = RandomForestClassifier()
model.fit(final_data, final_target)
validation_data = pd.read_csv('validate_nolabel.txt', sep=",")[[0,1]]
validation_data.columns = ["Q_id","U_id"]
final_data = pd.merge(question_data, validation_data, on="Q_id",how = 'inner')
final_data = pd.merge(final_data, expert_data, on='U_id', how='inner')
predicted = model.predict_proba(final_data)
outputdf = pd.DataFrame(pd.DataFrame(predicted)[1])
outputdf.columns = ["output"]
final_predicted_data = pd.concat([validation_data,outputdf],axis=1)
final_predicted_data.to_csv('val.csv',sep=',', index=False, header = False)

MemoryError: 